# Workflow to generate refractory alloy compositions and predict their strength

Steps in this notebook:

Training the models:

1) Train autoencoder to encode compositions and solidus temperatures into a 2D latent space
    Training data is Thermo-Calc equilibrium calculations with compositions randomly sampled from this element palette [Cr, Hf, Nb, Mo, Ta, Ti, Re, V, W, Zr]
    Thermo-Calc generated solidus temperature (Tsol), liquidus temperature (Tliq), and the fraction of ordered (N(OrdPh) and disordered (NDisordPh) phases at Tsol
    
2) Cluster the latent space by Tsol into four solidus temperature classes: 1000-1700, 1700-2400, 2400-3100, 3100-3800 K

3) Train a random forest model to predict alloy solidus temperature as a function of composition

4) Train a random forest regression model to predict alloy yield strength and ultimate tensile strength as a function of composition and temperature.
    Training data is from the literature.

Note:  There are saved versions of the autoencoder model in the '/alloy-main/model/' folder.  These models do not need to be retrained, see comments in cells below.

Using the models:

1) Sample points from clusters in the autoencoder latent space, this example draws from the 3100-3800K Tsolidus cluster

2) Decode with autoencoder to compositions with a specified range of Tsol

3) Predict solidus with random forest model

4) Predict yield strength and ultimate tensile strength with random forest model# Machine Learning for Refractory Alloy

Set `load_pretrain = False` if you want to train the models 

In [ ]:
load_pretrain_autoencoder = True # Default: True
load_pretrain_RF_solidus = False # Default: False

## Import Function

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error,accuracy_score,r2_score
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.optim import lr_scheduler
from tqdm import tqdm
import pickle


## Autoencoder

![fig1](./fig/fig1.png)

In [ ]:
# Load tc_data
tc_data = pd.read_csv('./data/tc_data.csv')

# elements input for autoencoder
tc_element = tc_data.iloc[:,:10] 
tc_data

### Plot the distribution of the Thermo-Calc data

In [ ]:
# Rank by average fraction
sort = tc_element.mean().sort_values(ascending=False).reset_index()
sort_column = sort['index']
tc_element = tc_element[sort_column]
elements = tc_element.columns.tolist()
print(elements)
x_index = np.zeros(len(tc_element))
# plt.figure(figsize=(8,8))

for i in elements:
    condition = tc_element[i] > 0.0
    plt.scatter(x_index[condition],tc_element[i][condition],alpha=0.01,s = 100,marker='o')
    x_index+=1

plt.plot(range(len(elements)),sort[0],'ko-',label='Average Concentration')
# plt.legend()
plt.xticks(range(len(elements)),elements)
plt.ylabel('Fraction')
plt.xlim(-1,len(elements))
plt.ylim(-0.1,1.1)

In [ ]:
tc_data.Tsol.hist(bins=100)
plt.xlabel('Solidus Temperature(K)')
plt.ylabel('Count')

### Set up Dataloader for Autoencoder

In [ ]:
# Create Dataset object 
class Alloy_Dataset(Dataset):
    def __init__(self, elements, labels):
        super().__init__()
        assert len(labels) == len(elements)
        self.elements = elements
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
      
    def __getitem__(self, index):
        input = self.elements[index]
        target = self.labels[index]
        return input , target

data = Alloy_Dataset(tc_element.values,tc_data['label']) # classification
# data = Alloy_Dataset(tc_element.values,tc_data['Tsol']) # regression

# train test split
train_ratio = 0.8
train_size = int(train_ratio * len(data))
test_size = len(data) - train_size
train_set, test_set = torch.utils.data.random_split(data, [train_size, test_size], generator=torch.Generator().manual_seed(42))

# Dataloader
train_loader = DataLoader(
    train_set, # The dataset
    batch_size=128,      # Batch size
    shuffle=True,      # Shuffles the dataset at every epoch
    pin_memory=True,   # Copy data to CUDA pinned memory                   # so that they can be transferred to the GPU very fast
    num_workers=0      # Number of worker processes for loading data.
                       # If zero, use the current process (blocks until data are loaded)
                       # Otherwise fork/spawn new processes (asynchronous load)
                       # Spawning new processes can be problematic on Windows, see:
                       # https://pytorch.org/docs/stable/notes/windows.html#usage-multiprocessing
                       )
test_loader = DataLoader(test_set,batch_size=128,shuffle=False,pin_memory=True,num_workers=0)

In [ ]:
# Check cuda 
device = "cuda" if torch.cuda.is_available() else "cpu"
# 'cuda: 0' if multiple GPUs
print(device)

## Architecture of Autoencoder

In [ ]:
# Autoencoder Model
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(10,24),
            nn.ReLU(True),
            nn.Linear(24,48),
            nn.ReLU(True),
            nn.Linear(48,64),
            nn.ReLU(True),
            nn.Linear(64,32),
            nn.ReLU(True),
            nn.Linear(32,16),
            nn.ReLU(True),
            nn.Linear(16,8),
            nn.ReLU(True),
            nn.Linear(8,4),
            nn.ReLU(True),
            nn.Linear(4,2)
            #nn.ReLU(True)
        )
        self.decoder = nn.Sequential(
            nn.Linear(2,4),
            nn.ReLU(True),
            nn.Linear(4,8),
            nn.ReLU(True),
            nn.Linear(8,16),
            nn.ReLU(True),
            nn.Linear(16,32),
            nn.ReLU(True),
            nn.Linear(32,64),
            nn.ReLU(True),
            nn.Linear(64,48),
            nn.ReLU(True),
            nn.Linear(48,24),
            nn.ReLU(True),
            nn.Linear(24,10),
            nn.ReLU(True),
        )
        self.classifier = nn.Sequential(
            nn.Linear(2,4)
        )

    def forward(self, input):
        en_out  = self.encoder(input)
        pred   = self.classifier(en_out)
        de_out  = self.decoder(en_out)
        return en_out,de_out, pred



## Set up for training the model

In [ ]:
# Put model into GPU if available
model = autoencoder().double().to(device)

# parameters set up
epochs = 300
loss_ratio = 2
criterion1 = nn.MSELoss()
criterion2 = nn.CrossEntropyLoss()
criterion3 = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler =  lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, threshold=0.001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=True)

### Training 

In [ ]:
if load_pretrain_autoencoder:
    pass
else:
    # trainning
    training_loss = []
    testing_loss = []
    pbar = tqdm(range(epochs))
    for i in pbar:
        epoch_loss = 0.0
        for input, target in train_loader:
            model.train()
            input = input.to(device)
            target = target.to(device)
            optimizer.zero_grad() 

            encode_output,decode_output, prediction = model(input)
            loss = criterion1(decode_output,input) + 2 * criterion2(prediction,target) # classification
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            pbar.set_postfix_str({'Epoch': i,'Training Loss': loss.item()})
        epoch_loss /= len(train_loader)
        training_loss.append(epoch_loss)
        scheduler.step(epoch_loss)
    plt.plot(range(len(training_loss)),training_loss)
    plt.title('Loss vs Epochs')
    plt.xlabel('epoch')
    plt.ylabel('Loss')

### Saving or Loading Autoencoder Model

In [ ]:
if load_pretrain_autoencoder:
    saving_path = './model/my_model.pth'
    model = autoencoder().double().to(device)
    if torch.cuda.is_available():
        model.load_state_dict(torch.load(saving_path),strict=True)
    else:
        model.load_state_dict(torch.load(saving_path,map_location=torch.device('cpu')),strict=True)
else:
    saving_path = './model/model.pth'
    torch.save(model.state_dict(),saving_path)


### Test the performance

In [ ]:
pred_list = []
target_list = []
encode_list = []
with torch.no_grad():
    for input, target in test_loader:
        model.eval()
        input = input.to(device)
        target_list += target.detach().cpu().numpy().tolist()
        encode_output,decode_output, prediction = model(input)
        encode_list += encode_output.detach().cpu().numpy().tolist()
        pred_list += prediction.detach().cpu().numpy().tolist()
    pred_list = np.argmax(pred_list,axis=1)
    encode_array=np.array(encode_list)
print(f'Accuracy of the Model is {accuracy_score(pred_list,target_list).round(5)}')

### Latent Space

In [ ]:
fig, ax = plt.subplots()
labels = ['1000 - 1700 K', '1700 - 2400 K', '2400 - 3100 K', '3100 - 3800 K']  # Custom labels for the points
scatter = ax.scatter(encode_array[:,0],encode_array[:,1], c= target_list,label = labels,s=10,cmap=plt.cm.get_cmap('Dark2',4))
# legend1 = ax.legend(*scatter.legend_elements(), loc = 'best',title = 'Classes')
plt.xlabel('Latent Vector1')
plt.ylabel('Latent Vector2')
# plt.legend(loc = 'lower left')
#plt.xlim(-3,10.5)
#plt.ylim(-1,1)
# plt.axes('off')
# ax.add_artist(legend1)
handles, _ = scatter.legend_elements()
legend1 = ax.legend(handles, labels, loc='center right', title='Classes',bbox_to_anchor=(1.4, 0.5))


## Random Forest Regressor for Solidus Temperature

![fig2](./fig/fig2.png)

In [ ]:
rf_solt = RandomForestRegressor(n_estimators=200,random_state=42)
X = tc_element
y = tc_data['Tsol']
X_train,X_test, y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
if load_pretrain_RF_solidus:
    print(f'Loading Model')
    with open('./model/RF_solidus.pkl','rb') as f:
        rf_solt = pickle.load(f)
else:
    print('Saving Model')
    rf_solt.fit(X_train,y_train)
    with open('./model/RF_solidus.pkl','wb') as f:
        pickle.dump(rf_solt,f)
y_pred = rf_solt.predict(X_test)
print(f'R^2 score = {r2_score(y_pred,y_test)}')
print(f'MAE = {mean_absolute_error(y_pred,y_test)}')

In [ ]:
y_pred_train = rf_solt.predict(X_train)
plt.plot(y_test,y_test,'r')
plt.scatter(y_test,y_pred,alpha=0.15,label='test')
plt.scatter(y_train,y_pred_train,alpha=0.15,label='train')
plt.xlabel('Solidus Temperature by Thermo-Calc')
plt.ylabel('Solidus Temperature by Random Forest')
plt.legend()

## Random Forest Regressor for Strength 

![fig3](./fig/fig3.png)

In [ ]:
# Load Data 
strength_name = 'YS'
strength = pd.read_csv('./data/ys_clean.csv')
include_tsol = True
# Including Solidus temperature if available

augment = True # Augmentation
ratio = True # Ratio of Tsol and T test
if include_tsol == False:
    strength.drop(columns=['Solidus temperature'],inplace=True)
elif ratio == True:
    strength['ratio'] = strength['Test temp']/strength['Solidus temperature']
strength

### Visulization of data distribution

In [ ]:
if include_tsol:
    if ratio:
        strength_element = strength.iloc[:,3:-1]
    else:
        strength_element = strength.iloc[:,3:]
else:
    strength_element = strength.iloc[:,2:]
# Rank by average fraction
sort = strength_element.mean().sort_values(ascending=False).reset_index()
sort_column = sort['index']
strength_element = strength_element[sort_column]
strength_element
elements = strength_element.columns.tolist()
print(elements)
x_index = np.zeros(len(strength))
# plt.figure(figsize=(8,8))

for i in elements:
    condition = strength_element[i] > 0.0
    plt.scatter(x_index[condition],strength_element[i][condition],alpha=0.04,s = 100,marker='o')
    x_index+=1

plt.plot(range(len(elements)),sort[0],'ko-',label='Average Concentration')
plt.legend()
plt.xticks(range(len(elements)),elements)
plt.ylabel('Fraction')
plt.xlim(-1,len(elements))
plt.ylim(-0.1,1.1)

In [ ]:
strength['Test temp'].hist(bins=100)
plt.xlabel('Testing Temperature(K)')
plt.ylabel('Count')

### Augmentation
Set `Strength = 0` at `Solidus Temperature` into train set. Prevent Strength at above alloy Solidus Temperature

In [ ]:
# Augmentation required solidus temperature
if augment == True:
       augmented_data = strength.copy()
       augmented_data['Test temp'] = augmented_data['Solidus temperature']
       if ratio ==True:
              augmented_data['ratio'] = augmented_data['Test temp']/augmented_data['Solidus temperature']

       augmented_data.drop(columns=[strength_name],inplace=True)
# augmented_data

### Train and Test

In [ ]:
X = strength.drop(columns=[strength_name])
y = strength[strength_name]

X_train,X_test, y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

if augment:
    X_train = pd.concat([X_train,augmented_data])
    y_train = y_train.tolist() + np.zeros([len(augmented_data)]).tolist()
    y_train = np.array(y_train)
rf = RandomForestRegressor(n_estimators=200,random_state=42)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
y_pred_train = rf.predict(X_train)
print(f'R^2 value of the model is {rf.score(X_test,y_test)}')
print(f'Mean absolut Error(MAE) of the model is {mean_absolute_error(y_pred,y_test)}')

fig, ax = plt.subplots()
plt.scatter(y_test,y_pred,label='Test',alpha=0.7)
plt.scatter(y_train,y_pred_train,label='Train',alpha=0.7)
plt.plot(y_train,y_train,'r',label='Pred = Target')
plt.legend()
plt.xlabel(f'True Value of {strength_name}[MPa]')
plt.ylabel(f'Prediction Value of {strength_name}[MPa]')

## Generate New Composition

![fig4](./fig/fig4.png)

In [ ]:
high_tsol = tc_data[tc_data['label']==3]
high_tsol

In [ ]:
tc_element = high_tsol.iloc[:,:-5]
sort = tc_element.mean().sort_values(ascending=False).reset_index()
sort_column = sort['index']
tc_element = tc_element[sort_column]
elements = tc_element.columns.tolist()
print(elements)
x_index = np.zeros(len(tc_element))
# plt.figure(figsize=(8,8))

for i in elements:
    condition = tc_element[i] > 0.0
    plt.scatter(x_index[condition],tc_element[i][condition],alpha=0.01,s = 100,marker='o')
    x_index+=1

plt.plot(range(len(elements)),sort[0],'ko-',label='Average Concentration')
# plt.legend()
plt.xticks(range(len(elements)),elements)
plt.ylabel('Fraction')
plt.xlim(-1,len(elements))
plt.ylim(-0.1,1.1)

### Sampling and Decoding

![fig5](./fig/fig5.png)


In [ ]:
high_tsol_elements = high_tsol.iloc[:,:-5]
high_tsol_tensor = torch.Tensor(high_tsol_elements.to_numpy()).double().to(device)
latent_high_tsol = model.encoder(high_tsol_tensor).detach().cpu().numpy()
new_composition_latent = []
# Generate sample
sample_number = 1000
for i in range(sample_number):
    a,b = np.random.choice(len(latent_high_tsol),2)
    new_x = (latent_high_tsol[a][0]  + latent_high_tsol[b][0])/2 
    new_y = (latent_high_tsol[a][1]  + latent_high_tsol[b][1])/2
    new_composition_latent.append([new_x,new_y])

# Decode to composition
new_composition = model.decoder(torch.Tensor(new_composition_latent).double().to(device))
new_composition = new_composition.detach().cpu().numpy()

# Scaled fraction into sum = 1.0
new_composition_df = pd.DataFrame(new_composition)
new_composition_df.columns = high_tsol_elements.columns
new_composition_df['sum'] = new_composition_df.sum(axis=1)
for x in high_tsol_elements:
    new_composition_df[x] /= new_composition_df['sum']
new_composition_df.drop(columns=['sum'],inplace=True)
new_composition_df

### Use RF_solidus to predict the Solidus Temperature for new composition

![fig6](./fig/fig6.png)


In [ ]:
new_composition_df['Solidus temperature'] = rf_solt.predict(new_composition_df)
# new_composition_df

![fig7](./fig/fig7.png)


In [ ]:
new_input = pd.concat([X_train.iloc[:1,:],new_composition_df])
new_input = new_input.iloc[1:,:]
new_input.fillna(0,inplace=True)
output_file = new_input.drop(columns=['Test temp'])
# Prediction for different Testing Temperature
T_test_list = np.arange(300,2500,100)
for t in T_test_list:
    new_input['Test temp'] = t
    if ratio:
        new_input['ratio'] = new_input['Test temp']/new_input['Solidus temperature']
    pred = rf.predict(new_input)
    output_file[f'{strength_name}_{t}K'] = pred
    plt.scatter(t*np.ones(len(pred)),pred)
plt.xlabel('Testing Temperature(K)')
plt.ylabel(f'{strength_name} (MPa)')

### Saving New Alloy Strength Prediction

In [ ]:
if ratio:    
    output_file.drop(columns=['ratio'],inplace=True)
output_file.to_csv(f'./output/new_alloy_{strength_name}.csv',index=None)
output_file